In [1]:
NAME = "Sarale Goldberger"
COLLABORATORS = ""

# Instructions

1. Make sure you have filled out your "NAME" and "COLLABORATORS" (if any) in the previous cell.

2. You should complete all code/markdown cells that state "YOUR CODE HERE" or "YOUR ANSWER HERE". 
   
3. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

4. Partial credit can be obtained if your solution approach is clear and the documented within comments in the implementation.

5. You should follow good coding practices. Your code should use type hints, be robust against invalid inputs, and you should also write a few test cases to check for correctness particularly including edge cases.  


# Supervised Learning

In this homework, we will build a few models from scratch and then use them to explore a real world dataset.  We are hoping to get some insight into the following topics:

1. How does the training process work?
2. What is the flow for the prediction process?
3. What does a model even look like?

Most machine learning libraries or packages will make some assumptions about the input and output format of the data.  Here we will also standardize on a format which is inspired by one such library.

We will assume that our data is structured as a dataframe with a special column called "label" for the classification target.  So for a problem with two features, it might look like:

|Feature 1|Feature 2|label|
|---|---|---|
|$x_{0,0}$|$x_{0,1}$|$y_{0}$|
|$x_{1,0}$|$x_{1,1}$|$y_{1}$|
|$\vdots$|$\vdots$|$\vdots$|
|$x_{n,0}$|$x_{n,1}$|$y_{n}$|

Please make sure that your code is passing all the `assert` statements before you move onto the next part. Do not assume however, that the `assert` statements will catch all cases, you will need to do your own testing as well.

# Problem 1  - Train a decision Tree

We start with building a decision tree classifier and implement both the training and inference.  As this is our first machine learning algorithm, we will take it slow and build it in many small steps and then put it all together.

Our goal will be to build up a decision tree model using a variant of the C4.5 algorithm. This algorithm is an extension of the ID3 algorithm and solves many of the limitations of that algorithm. 

For more materials on this algorithm, please peruse

- https://en.wikipedia.org/wiki/C4.5_algorithm


The first few parts of this algorithm will be to build up a decision tree training algorithm, the second parts are to explore what to do with it.

The algorithm proceeds as follows:

1. Start with a dataset.  
2. Check the base case for terminating recursion
3. Choose the splitting criteria which maximizes gain (i.e. which feature for which you want to split)
4. Split the dataset on this critera and recurse on each split

The root node of the decision tree sees the entire training dataset, whereas the child nodes will only see that subset of the entire dataset that results from the splitting of the dataset at each parent level in the decision tree. 

For simplicity, we will consider the base case for the dataset at a given node,  which will lead to the termination of the recursion:

1. All the examples in the dataset have the same label
2. The number of examples in the dataset is less than or equal to MAX_EXAMPLES_PER_NODE, whose value can be set to some small fixed number (say 3). 

Lets start by building up some of the helper functions which we will need as part of the algorithm.

## Part A

First we will need to define a function to compute the entropy of a pandas series comprising the label column in the dataset. we define entropy as 

$$H(D) =  \sum_i - p_i \ln_2(p_i)$$

where the summation index $i$ is over the unique labels in the label column, and $p_i$ is the proportion of the $i$'th label in the target column of the dataset.  For any data set the entropy can be calculated from the training data quite easily. Be careful with log evaluations in this expression, and note that $p_i \\ln_2(p_i) = 0$ for $p_i = 0$ (which acn be verified by evaluating this expression in the limit $p_1 \rightarrow 0$  via L'Hospital's rule).

For the case of a dataset, the probability of each element occuring can be calculated from the training data quite easily. 

For the sake of checking your initial answers please use the following dataset and define assertions for testing the code.

In [2]:
import pandas as pd
trial_df = pd.DataFrame({
    'a': [1, 2, 3, 4, 2],
    'b': ['cat', 'dog', 'cat', 'dog', 'lion'],
    'label': [True, False, True, False, True]
})
trial_df

,a,b,label
0,1,cat,True
1,2,dog,False
2,3,cat,True
3,4,dog,False
4,2,lion,True


In [3]:
# import numpy so we can use the almost equal method
import numpy as np
def assert_almost_equal(a, b):
    """assert almost equal with readable error message"""
    assert np.isclose(a, b), f"{a} != {b}"

In [4]:
def freq(s:pd.Series) -> dict:
    # Compute the frequency of each unique element
    freq_dict = {}
    for data in s: 
        freq_dict[data] = freq_dict.get(data, 0) + 1
    return freq_dict

def probability(s:pd.Series) -> dict:
    # Compute the frequency of each unique element
    freq_dict = freq(s)
    # Calculate the probability distribution
    for k in freq_dict:
        freq_dict[k] = freq_dict[k]/len(s)
    return freq_dict

def entropy(s: pd.Series) -> float:
    en = 0
    prob = probability(s)
    for p in prob.values():
        en += -p*np.log2(p)
    return en

In [5]:
assert_almost_equal(entropy(trial_df['a']), 1.9219280948873623)

## OLD Part B

### Part B.1
Now we consider the gain, which you can think of as the information that you gain from making a decision for a particular variable.  Intuitively, we want to choose the variable to make a split which will give us the most information once we choose.  In order to do this, we look at the difference in entropy between the full problem and the subset of the problem which would occur if we take a particular value as a given. 

Not that the potential split conditions you evaluate for each feature will depend on whether $ftr$ is a numerical or categorical feature, as follows

$$ G(D, ele) = H(D) - \sum P(D|ele)H(D|ele)$$
where $H$ is entropy $D$ is the dataset and $(D|ele)$ is the dataset given a chosen element. $H(D)$ is the entropy of the dataset given the label column.

The nature of the feature (either numerical or categorical) can be obtained from the dtype of the feature in the dateset.

The return value from this function will be the tuple with the first element as max_gain value over all the splits in the feature, and the second element being a List containing either the split value (for a numerical feature, or the set of values in the first set of the split for the categorical feature).   

In [6]:
def gain(df: pd.DataFrame, ele: str, label_key: str) -> float:
    sum = 0
    for data in df[ele].unique():
        subset = df[ df[ele] == data ]
        sum += len(subset)/len(df[ele]) * entropy(subset[label_key])
    return entropy(df[label_key]) - sum

In [7]:
assert_almost_equal(gain(trial_df, 'b', 'label'), 0.9709505944546686)

### Part B.2

Now we can compute the gain ratio for a dataframe.  The gain ratio is defined as 

$$ GR = \frac{G}{SI}$$

Where $SI$ is the split information defined as 
$$ -\sum_i \frac{N_i}{|N|}\log_2\left(\frac{N_i}{|N|}\right)$$
where $N_i$ is the number of occurencies of a unique value for a particular element and $|N|$ is the total number of elements in that dataset.

In other words, I have a dataset like 

|Color of Shirt|
|--------|
| red|
|blue|
|red|
|blue|
|blue|

Then the split info would be computed as 

$$ SI(\text{Color of Shirt}) = -\left[\frac{N_{red}}{|N|}\log_2\left(\frac{N_{red}}{|N|}\right) + \frac{N_{blue}}{|N|}\log_2\left(\frac{N_{blue}}{|N|}\right)\right] = - \left[\frac{2}{5}\log_2\left(\frac{2}{5}\right) + \frac{3}{5}\log_2\left(\frac{3}{5}\right)\right]\approx 0.971$$

Please feel free to leverage the following resources:

- https://machinewithdata.com/2018/07/10/how-to-calculate-gain-ratio/

Please define a function `gain_ratio(df, ele, label_key)` which computes the gain ratio of column `ele` in dataframe `df` where the label column name is given as `label_key`.

In [8]:
def gain_ratio(df: pd.DataFrame, ele: str, label_key: str) -> float:
    freq_dict = freq(df[ele])
    SI = 0
    for data in freq_dict:
        p = freq_dict[data]/len(df[ele])
        SI += p*np.log2(p)
    if SI == 0: SI = 1
    return gain(df, ele, label_key)/(-SI)

In [9]:
assert_almost_equal(gain_ratio(trial_df, 'b', 'label'), 0.6379740263133316)

### Part B.3

So far we have only handled categorical variables, we would like to also handle numerical variables.  What we will do is create a categorial variable out of our numeric variables by create a point $p$ and then each point is either $<=p$ or $>p$.  Once we create this point, we have a categorical feature of True, False values.  The trick here is how to compute the correct $p$.  The way we will do this is as follows:

1.  What we do is for each value of the attribute, we split the dataset into all values less than or equal to that value and all values greater than that value.  
2. We then compute the gain ratio as normal.  
3. Take the $p$ which maximizes the gain ratio and use that in our fit.

We now loop over all the features, calling `max_gain_for_feature`, and find the feature which leads to the maximum gain. 

In [10]:
def max_numeric_gain_ratio(df: pd.DataFrame, ele: str, label_key: str):
    ele_uniq = df[ele].unique()
    
    # Initialize max gain ratio and corresponding threshold value
    max_GR = 0
    p = ele_uniq[0]
    
    # try different values of p
    # compute the gain ration for each of the unique elements of df[ele] as p
    # save the max gain ratio and corresponding p value
    for data in ele_uniq:
        df['over_p'] = df[ele] > data
        GR_temp = gain_ratio(df, 'over_p', label_key)
        if GR_temp > max_GR: 
            max_GR = GR_temp
            p = data
            
    return (max_GR, p)

In [11]:
res = max_numeric_gain_ratio(trial_df, 'a','label')
assert_almost_equal(res[0],  0.4459281986214854)
assert res[1] == 3, f"{res[1]} != 3"

## NEW Part B

Now we consider the information gain, which you can also equivalently think of as the reduction in entropy from making a binary split decision for a particular feature.  Intuitively, we want to choose the feature  which will give us the larget information gain (equivalently the largest entropy reduction) once we choose it to split the data set.  In order to do this, we first evaluate the information gain  between the full dataset and the two subsets obtained by  splitting the dataset using all posiblle conditions for a given feature column $ftr$.  We can then loop over all feature columns in this way to find the best choice of feature column (and the corresponding split condition using that column).

Mathematically, if the feature $ftr$ is used to split the dataset $D$ (with $N$ examples) into two datasets $D_1$ and $D_2$ (with $N_1$ and $N_2$ examples respectively, then the Information Gain from this split  is denoted by

$$ G(D, ftr) = H(D) - \\[\\frac{N_1){N}H(D_1) + \\frac{N_2){N}H(D_2) \\]$$. 
    
Please define a function `max_gain_for_feature(df, ftr, label_key)` which computes the largest gain of spliting the dataset using the values in the column `ftr` in dataframe `df` where the label column name is given as `label`.

Not that the potential split conditions you evaluate for each feature will depend on whether $ftr$ is a numerical or categorical feature, as follows

- For numerical feature consider a set of potential split conditions as follows.  Le $k_1 < k_2, \\ldots k_J$ be the $J$ unique values in ascending order in the column and consider all splits of the form $(x \\leq k_i, x > k_i)$ for $i = 1, 2, \\ldots, J-1$.
- For categorical features consider a set of potential split conditions as follows: Let (a, b, c, d) be the unique values of the feature column in some initial arbitrary order, and consider the 3 splits of the form [(a), (b,c,d)], [(a,b), (c,d)], [(a,b,c), (d)]  In general if there are k unique values in some initial arbitrary order then there will be (k-1) split conditions.

The nature of the feature (either numerical or categorical) can be obtained from the dtype of the feature in the dateset.

The return value from this function will be the tuple with the first element as max_gain value over all the splits in the feature, and the second element being a List containing either the split value (for a numerical feature, or the set of values in the first set of the split for the categorical feature).

In [1]:
def max_gain_for_feature(df: pd.DataFrame, 
         ftr: str, 
         label: str) -> (float, List):
    # YOUR CODE HERE
    raise NotImplementedError()

NameError: name 'pd' is not defined

## Part C

We now loop over all the features, calling `max_gain_for_feature`, and find the feature which leads to the maximum gain.

In [ ]:
def find_best_split condition(df: pd.DataFrame,  
                              label: str) -> (str, List):
    # YOUR CODE HERE
    raise NotImplementedError()

## Part D

Lets start by implementing the simple id3 algorithm which will give us a good sense of how to do the more complex algorithm.

We will make a fake dataset of 10 observations with a label of true false for how well someone sleeps.

In [12]:
id3_data = pd.DataFrame({
    'day_type': ['long', 'long', 'short', 'medium', 'short', 'short', 'medium', 'long', 'short', 'short'],
    'weekend': [True, False, False, False, False, False, True, True, True, False],
    'good_night_before': [True, True, False, True, True, False, False, False, False, True],
    'label': [True, False, False, False, True, False, True, False, True, False]
})
id3_data

,day_type,weekend,good_night_before,label
0,long,True,True,True
1,long,False,True,False
2,short,False,False,False
3,medium,False,True,False
4,short,False,True,True
5,short,False,False,False
6,medium,True,False,True
7,long,True,False,False
8,short,True,False,True
9,short,False,True,False


Next we will define an `Id3Node` class which will be used to hold the data for our algorithm.  Each of these nodes will be the node in a tree.

In [46]:
from dataclasses import dataclass
from typing import Dict, Optional
@dataclass
class Id3Node:
    key: str
    children: Optional[Dict[str, 'Node']] = None
    
    def insert(self, kids: [str]):
        # if children never initialized, initialize it to a dictionary
        if self.children == None:
            self.children = {}
        # if just one child to insert
        if type(kids) == str:
            self.children[kids] = Id3Node(key=kids)
        else:
            # add the chidlren to the dictionary
            for child in kids:
                self.children[child] = Id3Node(key=child)

    def __str__(self, node="start", b=0, ans=''):
        buffer = "–––>"
        if node == "start": node = self
        ans += node.key
        # if the child has children, print each of the children
        if node.children: #or len(self.children) > 0
            for child_key in node.children:
                # append buffer and child to answer string
                ans += f"\n{' '*b}{buffer}{node.__str__(node.children[child_key], b+len(buffer))}"
        return ans

In [47]:
root = Id3Node('root')
root.insert(['child1', 'child2', 'child3'])
root.children['child1'].insert(['child1a', 'child1b'])
print(root)

root
–––>child1
    –––>child1a
    –––>child1b
–––>child2
–––>child3


Now we can implement the `train_id3` method.  This method should take a dataframe and return an `Id3Node` which represents the trained tree.  

It is quite likely (although not strictly required) that your algorithm be recursive in nature.  Like any recursive algorithm, play close attention to the base cases defined above.  

In [122]:
def __str__(self, node="start", b=0, ans=''):
    if node == "start": node = self
    ans += node.key
    # if the child has children, print each of the children
    if node.children: #or len(self.children) > 0
        for child_key in node.children:
            # append buffer and child to answer string
            ans += node.__str__(node.children[child_key], b+len(buffer))
    return ans

def build_tree(df: pd.DataFrame, ele: str, root: Id3Node) -> Id3Node:  #, root: str
    # Create a node for each unique value in the data
    # Base case: we are at the label column. 
    return

def train_id3(df: pd.DataFrame, label_key = 'label', root = Id3Node('root')) -> Id3Node:
    print('cur:', root)
    # Base case: df is empty
    if len(df) == 0:
        print('end')
        return root
    
    # Base case: we only have the label column. 
    # if len(df) == 1 and df.columns[0] == label_key:
        # print(len(df))
        # print(df)
        # for data in df[label_key].unique():
        #     root.insert(f"{label_key}_{data}")
        # return root
        
    max_gain = 0
    best_ele = df.columns[0]
    # for each col in the df except the last, compute the gain function and find the best gain    
    for ele in df.loc[:, df.columns!=label_key]: 
        gain = gain_ratio(df, ele, df.columns[-1])
        if gain > max_gain:
            max_gain = gain
            best_ele = ele

    for data in df[best_ele].unique():
        root.insert(f"{best_ele}_{data}")
        # get a subset of the df where all the rows match data, and don't include the best_ele column
        # train on this subset and add as child to this node
        train_id3(df.loc[df[best_ele] == data, df.columns!=best_ele], label_key, root.children[f"{best_ele}_{data}"])
    
    # split the data based on the best column
    # NOTE: need another function for best split value within the column
    # build_tree(df, best_ele, root) #, df[best_ele].unique()[-1]
    
    return root

In [123]:
# for col in id3_data: print(col)
# first_col = id3_data.columns[0]
# print(id3_data[first_col].unique()[-1])
# for i in id3_data['weekend'].unique():
#     print(i)

# Test base case train_id3
id3_tree = train_id3(id3_data)
print(id3_tree)

# display(id3_data.loc[id3_data['weekend'] == True, id3_data.columns!='weekend'])

cur: root
cur: weekend_True
cur: day_type_long
cur: good_night_before_True
cur: label_True


IndexError: index 0 is out of bounds for axis 0 with size 0

Now we can create a `predict_id3` method which takes in a single row of a `DataFrame` (which is a `Series`) and a trained `Id3Node` (the root node) to make a prediction.

The prediction should take the row and walk down the tree at each step choosing the proper child given the row information until it gets to a leaf.

In [ ]:
def predict_id3(row: pd.Series, node: Id3Node):
    # YOUR CODE HERE
    raise NotImplementedError()

Now put it all together, fill in the `fit` and `predict` methods for the `Id3Model` class.

In [ ]:
class ModelNotTrainedError(ValueError):
    """model is not trained yet"""
        
class Id3Model:
    def __init__(self):
        self.tree = None
    
    def fit(self, df):
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def predict(self, df):
        # YOUR CODE HERE
        raise NotImplementedError()

## Part E (BONUS)

This is a bonus part to the problem.  It is not trivial to implement, but I encourage you to try!  Partial or correct solutions to this part will be worth some extra credit.

We have already implemented the ID3 algorithm, now you can use the functions before to implement the same for the numeric types.

Now we can put it all together, implement a training algorithm which produces a trained tree. To help you, we will define a class `Node` as well as a visualize function to produce an image of the node, you can choose to use your own data structure if you so wish.

In [ ]:
from dataclasses import dataclass
from typing import Dict
@dataclass
class Node:
    key: str
    numeric: bool
    pivot: float
    children: Dict[str, 'Node']

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part F

Now use this algorithm to fit the wine dataset below.  Give an overview of your results and an explanation of your findings.  If you did not do the bonus problem in part e), you may use the class below or the `scikit-learn` model directly.  The class is a very simple wrapper to conform to the dataframe format we have been using.


Some questions to consider:

1. Do your results make sense?

**Note**: It may take some time to train your algorithm

In [ ]:
from sklearn.tree import DecisionTreeClassifier
class DecisionTreeModel(DecisionTreeClassifier):
    
    def fit(self, df):
        if 'label' not in df:
            raise ValueError("Label is not in df")
        X = df.drop('label', axis=1)
        self.columns_ = X.columns.tolist()        
        super().fit(X.values, df['label'].values)
        return self
    
    def predict(self, df):
        X = df[self.columns_].values if isinstance(df, pd.DataFrame) else df
        return super().predict(X)

In [ ]:
from sklearn.datasets import load_wine
import pandas as pd
dataset = load_wine()
df = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
df['label'] = dataset['target']
df.head()

# Problem 2

In this problem, we will implement the perceptron algorithm as defined in the book, below is the skeleton class we will want to implement.

Your alogorithm should take as a hyperparametr `max_iters` which is the number of times it will iterate through the training set entirely.

Please include a `get_params` method which returns a `PerceptronParams` class with the current parameters of the model.  Its your choice how you store these, but this method must return this object type as it will be used to validate your results.

For consistency with the results, please initialize all weights and biases to zero.

In [ ]:
from dataclasses import dataclass
from typing import List
@dataclass
class PerceptronParams:
    weights: List[float]
    bias: float

class Perceptron:
    def __init__(self, max_iters=20):
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def get_params(self) -> PerceptronParams:
        # YOUR CODE HERE
        raise NotImplementedError()
    def fit(self, df, callback=None):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, df):
        # YOUR CODE HERE
        raise NotImplementedError()

## Part A

Now lets explore linear separability with the same Iris dataset that was used in Lecture 4.  Check that lecture to see how the data was preprocessed for a binary classification (setosa versus non-setosa) with just 2 features (petal width and petal length, and replicate that dataset in the next code block



In [ ]:
# YOUR CODE HERE

Now train a perceptron and plot the best fit line on top of the scatter plot from above.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Now lets examine how the line evolves as the model is trained.  Make a plot showing how the line changes per iteration.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part B (Bonus - extra credit)

For extra credit, implement averaging in the perceptron model.

In [ ]:
class AveragePerceptron(Perceptron):
    # YOUR CODE HERE
    raise NotImplementedError()

Now make the same plot as in the previous section.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part C

Analyze the stability of the weights as a function of iteration, If you did part b, please include it in your analysis.  Some points to consider

- How stable were they over time?
- How many iterations were appropriate?

YOUR ANSWER HERE

# Problem 3 - Conceptual

For this problem, write each answer in the cell immediately following the question which will be marked.

## Part A

The algorithm we have used to train the decision tree is a greedy algorithm.  Explain why we use a greedy algorithm and what consequences this has on the resulting model.

YOUR ANSWER HERE

## Part B

In our perceptron algorithm, *IN YOUR OWN WORDS*, why is it important to shuffle the datasets?  

YOUR ANSWER HERE

## Part C

Is a decision tree guaranteed to find a globally optimal solution?  If not, what are the barriers to creating an algorithm to find the globally optimal solution.

YOUR ANSWER HERE

# Problem 4 - Data Problem

Problem 4 is the real world simulation problem.  Here I will simply give you a dataset and a problem, your goal is to solve the problem and list all of your assumptions as well as your results.  This is meant to simulate many of the types of problems you may see in the future on an interview.

You will be evaluated on how well you use the techniques we have learned so far in the course, you will not need to have the best model, you will be evaluated more on how you think and explain your solution.

Here we are going to use the very common dataset, california housing.


## Part A

First of all do some exploratory data analysis and report your findings.  The following function will get the data for you, please take it from there!

In [ ]:
from sklearn.datasets import fetch_california_housing

blob = fetch_california_housing()
df = pd.DataFrame(blob['data'], columns=blob['feature_names'])
df[blob['target_names'][0]] = blob['target']

In [ ]:
# display(df)
# df.info()
df.describe()

## Part B

Before modeling, its often good to start with a baseline model, something simple which will give us some indication if the model we are building is actually learning something.  Lets start by building a model which predicts that the median housing price of a block is the median of the entire dataset.  Please compute the mean squared error of this model

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part C

In this problem we are going to use a linear regression to explore this dataset.  We will learn more about scikit-learn in the future, here you will be provided with an object to fit a dataframe to a linear model.  You are welcome to use the `scikit-learn` object directly if you so choose.

In [ ]:
from typing import List, Optional
from sklearn.linear_model import LinearRegression

class LinearModel:
    def __init__(self, fit_intercept: bool = True):
        self.fit_intercept = fit_intercept
        self.target_variable: Optional[str] = None
        self.columns: Optional[List[str]] = None
        self.model: Optional[LinearRegression] = None
        
    def _check_fit(self):
        if self.model is None:
            raise RuntimeError("model is not yet fit")
        
    def fit(self, df: pd.DataFrame, target_variable: str) -> 'LinearModel':
        """fit a dataframe and return the coefficients and intercept
        
        Parameters
        ----------
        df: pd.DataFrame
            Input dataframe for fitting, should contain target variable
        target_variable: str
            Target variable for fitting, must be in the dataframe
        """
        self.target_variable = target_variable
        X = df.drop(target_variable, axis=1)
        y = df[target_variable].values
        self.columns = X.columns
        self.model = LinearRegression(fit_intercept=self.fit_intercept)
        self.model.fit(X.values, y)
        return self
    
    @property
    def coef(self):
        self._check_fit()
        return dict(zip(self.columns, self.model.coef_))
    
    @property
    def intercept(self) -> float:
        self._check_fit()
        return self.model.intercept_

    
    def predict(self, df: pd.DataFrame):
        """
        Make predictions using the fitted model
        
        Parameters
        ----------
        df: pd.DataFrame
            Input dataframe for prediction
        """
        self._check_fit()
        X = df[self.columns].values
        return self.model.predict(X)

Start off by fitting the linear regression directly to the dataset and then interpret your results.  Remove the Latitude and Longitude information for now (more on this in the next part).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part D

Latitude and Longitude are different sorts of features than the rest of the features in this dataset, please explain why.

**Hint**: Try making a plot

ANSWER: They only make sense together.

## Part E (Bonus)

Now we are going to use the latitude and longitude to generate some new features.  Often in an ML problem, bringing more is how we can best improve our predictive accuracy.

Instead of using directly, lets create a feature which is minimum distance from a "major" city, defined as having greater than .5 million people.

I have looked up the following wikipedia:

| City | Latitude | Longitude |
| --- | ---| ---|
|Los Angeles| 34.03| -118.15|
|San Diego|32.4254| -117.0945|
|San Francisco|37.4639| -122.2459|
|San Jose |37.2010| -121.5326|
|Sacramento|38.3454| -121.2940|

You are welcome to use any distance metric you like, however, one good one would be from the geopy library. 

In [ ]:
lat_longs = [
    (34.04, -118.15),
    (32.4254, -117.0945),
    (37.4639, -122.2459),
    (37.2010, -121.5326),
    (38.3454, -121.2940)
]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part F

Now please create the best linear model that you can, produce a explanation of the decision you made and why this is a "good" model.